# Imports

In [10]:
import mne
from mne.externals.pymatreader import read_mat
import numpy as np
import xarray as xr
import seaborn as sns
import pandas as pd
from tqdm.notebook import tqdm
import os.path as op
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import scipy
from mpl_toolkits.axes_grid1.inset_locator import InsetPosition
import sympy as sp
import matplotlib.image as mpimg
from pathlib import Path
import numpy.fft as fft
import matplotlib.image as mpimg

from acareeg.lemon import get_epochs_lemon, get_trials_lemon
from acareeg.utils import get_epochs_to_trials
from acareeg.simulation import get_epochs_sim, get_corrected_sources_one_epoch, \
    get_model_kdtree, correct_source_amplitude

# Functions

In [2]:
def get_dist2(eeg_loc, name1, name2):
    return np.sqrt(np.sum((eeg_loc.loc[name1].values - eeg_loc.loc[name2].values)**2, axis=1)) 

def get_coh_dist_df(trials_all, locations, con_method = "cohy", sfreq=250, between=False):

    if between:
        trials_all = trials_all.copy()
        tmp = trials_all.data
        for i in range(len(trials_all.channel)):
            tmp[:, i] = np.roll(tmp[:, i], shift=i, axis=2)            
        trials_all.data = tmp

    return_df = []
    for subject in tqdm(trials_all.subject):
        trial_subject = trials_all.sel(subject=subject, channel=locations[locations.subject == subject].index.values)
        con, freqs, times, n_epochs, n_tapers = \
            mne.connectivity.spectral_connectivity(trial_subject.data, 
                                                   method=con_method, sfreq=sfreq,
                                                   faverage=True, block_size=1000, verbose=False)
        ch_names = trial_subject.channel.values
        nb_signals = len(ch_names)
        inds = np.tril_indices(nb_signals, k=-1)

        ch_name1 = np.ravel(np.tile(ch_names, (nb_signals, 1))[inds])
        ch_name2 = np.ravel(np.tile(np.array(ch_names)[:, None], (1, nb_signals))[inds])

        con_df = pd.DataFrame({"con": np.ravel(con[inds]),
                                   "ch_names1": ch_name1, 
                                   "ch_names2": ch_name2})

        con_df["dist"] = get_dist2(locations[locations.subject == subject][["x", "y", "z"]], 
                                   con_df.ch_names1.values, con_df.ch_names2.values)*1000
        con_df["abs_cohy"] = np.abs(con_df.con)
        return_df.append(con_df)
    
    return pd.concat(return_df)

def get_sources(epochs, fwd, lambda2=1e-4, loose=0.0, spatial_random=False, phase_random=False, seed=4234): 
    noise_cov = mne.compute_covariance(epochs, method="auto", verbose=False).as_diag()
    inverse_operator = mne.minimum_norm.make_inverse_operator(epochs.info, fwd, noise_cov, loose=loose, verbose=False)     
    stcs = mne.minimum_norm.apply_inverse_epochs(epochs, inverse_operator, lambda2, method="eLORETA",
                                                 pick_ori=None, return_generator=False, verbose=False)
    if seed is not None:
        np.random.seed(seed)    
    if spatial_random or phase_random:
        for stc in stcs:
            if spatial_random:
                # Shuffling the sources                
                nb_sources = fwd["src"][0]["nuse"] + fwd["src"][1]["nuse"]
                inds = np.arange(nb_sources)
                np.random.shuffle(inds)                
                stc.data = stc.data[inds]
            if phase_random:
                stc.data = rand_phase(stc.data)

    return stcs

def get_vertice_pos(fwd):
    return np.vstack((fwd["src"][0]['rr'][fwd["src"][0]["vertno"]], 
                      fwd["src"][1]['rr'][fwd["src"][1]["vertno"]]))   

    
def sources_to_df(stcs, fwd, nb_selected_sources=100, con_method = "cohy", seed=34345345): #, nb_selected_epochs=10):
    # Selecting a subset of sources
    if seed is not None:
        np.random.seed(seed)
    selected_sources = np.random.choice(len(range(stcs[0].data.shape[0])), nb_selected_sources, False)
    selected_stcs = np.stack([s.data[selected_sources, :] for s in stcs])    
       
    # Computed coherency
    con_sources, freqs, times, n_epochs, n_tapers = \
            mne.connectivity.spectral_connectivity(selected_stcs, method=con_method, 
                                                   sfreq=epochs.info["sfreq"],
                                                   faverage=True, verbose=False)
    
    # Building the dataframes
    inds = np.tril_indices(nb_selected_sources, k=-1)

    ch_name1 = np.ravel(np.tile(selected_sources, (nb_selected_sources, 1))[inds])
    ch_name2 = np.ravel(np.tile(np.array(selected_sources)[:, None], (1, nb_selected_sources))[inds])    
    
    df_within_sources = pd.DataFrame({"con": np.ravel(con_sources[inds]),
                                      "ch_names1": ch_name1, 
                                      "ch_names2": ch_name2})

    pos = get_vertice_pos(fwd)
    df_within_sources["dist"] = get_dist2(pd.DataFrame(pos), df_within_sources.ch_names1.values, df_within_sources.ch_names2.values)*1000
    df_within_sources["abs_cohy"] = np.abs(df_within_sources.con)
    
    return df_within_sources    
    
def get_coh_dist_df_sources(epochs, fwd, nb_selected_sources=100, con_method = "cohy", 
                            seed=234234, **kwargs): #, nb_selected_epochs=10):
    stcs = get_sources(epochs, fwd, seed=seed, **kwargs)
    df_within_sources = sources_to_df(stcs, fwd, nb_selected_sources=100, con_method = "cohy", seed=seed)
    return df_within_sources, stcs

def get_epochs_sim_lemon(subject, fwd, lambda2=1e-4, spatial_random=False, phase_random=False, seed=4234):
    epochs = get_epochs_lemon(subject, lemon_root, event_id)
    epochs = epochs[nb_selected_epochs]
    epochs.load_data()
    if seed is not None:
        np.random.seed(seed)
    df_within_sources_01, stcs_01 = get_coh_dist_df_sources(epochs, fwd, lambda2=lambda2, spatial_random=spatial_random, 
                                                            phase_random=phase_random, seed=None)    
    epochs_sim = get_epochs_sim(epochs, stcs_01, fwd, return_snr=False)

    return epochs_sim, epochs, df_within_sources_01, stcs_01

def get_corrected_amplitude_surrogate(epochs, stcs, fwd, return_con_df=True, seed=34234):    
    correct_source_amplitude(epochs, stcs, fwd)
        
    # Re-simulate the EEG from the sources with corrected amplitude
    raw_sim = mne.simulation.simulate_raw(info=epochs.info, stc=stcs, forward=fwd, verbose=False)    
    epochs_sim = mne.Epochs(raw_sim, events, tmin=epochs.times[0], tmax=epochs.times[-1], 
                            baseline=None, preload=True, reject_by_annotation=False, verbose=False)    
    
    if return_con_df:
        return epochs_sim, sources_to_df(stcs, fwd, nb_selected_sources=100, seed=seed)    
    
    return epochs_sim

# Setting paths and variables

In [3]:
# This dataset is available online: http://fcon_1000.projects.nitrc.org/indi/retro/MPI_LEMON.html
lemon_root = Path("/media/christian/MyBook/Experimental_data/MPI-Leipzig_Mind-Brain-Body-LEMON")
eeg_rel_path = "EEG_MPILMBB_LEMON/EEG_Preprocessed_BIDS_ID/EEG_Preprocessed"
path_eeg = lemon_root / eeg_rel_path
glob_pattern = "sub-*_E?.set"
parse_pattern = "sub-{}_{}.set"
parse_pattern_montage = "sub-{subject}/sub-{subject}.mat"

nb_selected_epochs = np.arange(10)
event_id= "EO"

# Listing subjects with EEG, T1, FLAIR and electrode digitization

In [4]:
complete_subjects = []
for path in tqdm(list(path_eeg.glob(f"sub-*_{event_id}.set"))):
    subject = Path(path).name[4:-7]
    if ((lemon_root / f"BIDS_LEMON/sub-{subject}/ses-01/anat/sub-{subject}_ses-01_acq-mp2rage_T1w.nii.gz").exists() and
        (lemon_root / f"BIDS_LEMON/sub-{subject}/ses-01/anat/sub-{subject}_ses-01_acq-highres_FLAIR.nii.gz").exists() and
        (lemon_root / f"EEG_MPILMBB_LEMON/EEG_Localizer_BIDS_ID/sub-{subject}/sub-{subject}.mat").exists()):
        epochs = get_epochs_lemon(subject, lemon_root, event_id)
        if epochs.info["sfreq"] == 250:
            complete_subjects.append(subject)
            
complete_subjects = complete_subjects

/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these

/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these

/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these

/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these

/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these

/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these

# Building a forward model (head model) for the fsaverage subject from MNE-Python

In [5]:
epochs = get_epochs_lemon(complete_subjects[0], lemon_root, event_id)

fs_dir = mne.datasets.fetch_fsaverage(verbose=True)
subjects_dir = op.dirname(fs_dir)
subject = 'fsaverage'
trans = 'fsaverage'  # MNE has a built-in fsaverage transformation
src = op.join(fs_dir, 'bem', 'fsaverage-ico-5-src.fif')
bem = op.join(fs_dir, 'bem', 'fsaverage-5120-5120-5120-bem-sol.fif')

fwd = mne.make_forward_solution(epochs.info, trans=trans, src=src,
                                bem=bem, eeg=True, mindist=5.0, n_jobs=1)

0 files missing from root.txt in /usr/local/freesurfer/subjects
0 files missing from bem.txt in /usr/local/freesurfer/subjects/fsaverage
Source space          : /usr/local/freesurfer/subjects/fsaverage/bem/fsaverage-ico-5-src.fif
MRI -> head transform : /home/christian/code/mne-python/mne/data/fsaverage/fsaverage-trans.fif
Measurement data      : instance of Info
Conductor model   : /usr/local/freesurfer/subjects/fsaverage/bem/fsaverage-5120-5120-5120-bem-sol.fif
Accurate field computations
Do computations in head coordinates
Free source orientations

Reading /usr/local/freesurfer/subjects/fsaverage/bem/fsaverage-ico-5-src.fif...


/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)


Read 2 source spaces a total of 20484 active source locations

Coordinate transformation: MRI (surface RAS) -> head
     0.999994  0.003552  0.000202      -1.76 mm
    -0.003558  0.998389  0.056626      31.09 mm
    -0.000001 -0.056626  0.998395      39.60 mm
     0.000000  0.000000  0.000000       1.00

Read  61 EEG channels from info
Head coordinate coil definitions created.
Source spaces are now in head coordinates.

Setting up the BEM model using /usr/local/freesurfer/subjects/fsaverage/bem/fsaverage-5120-5120-5120-bem-sol.fif...

Loading surfaces...

Loading the solution matrix...

Three-layer model surfaces loaded.
Loaded linear_collocation BEM solution from /usr/local/freesurfer/subjects/fsaverage/bem/fsaverage-5120-5120-5120-bem-sol.fif
Employing the head->MRI coordinate transform with the BEM model.
BEM model fsaverage-5120-5120-5120-bem-sol.fif is now set up

Source spaces are in head coordinates.
Checking that the sources are inside the surface and at least    5.0 mm away (w

In [6]:
trials = []
trials_csd = []
for subject in tqdm(complete_subjects):
    path = path_eeg / f"sub-{subject}_{event_id}.set"
    trials.append(get_trials_lemon(subject, lemon_root, [event_id], use_csd=False).sel(epoch=nb_selected_epochs))
    trials_csd.append(get_trials_lemon(subject, lemon_root, [event_id], use_csd=True).sel(epoch=nb_selected_epochs))
trials = xr.concat(trials, dim="subject")
trials_csd = xr.concat(trials_csd, dim="subject")

/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these

Fitted sphere radius:         102.9 mm
Origin head coordinates:      0.4 3.3 48.6 mm
Origin device coordinates:    0.4 3.3 48.6 mm


/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these

Fitted sphere radius:         99.4 mm
Origin head coordinates:      4.3 5.8 49.5 mm
Origin device coordinates:    4.3 5.8 49.5 mm


/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these

...

Fitted sphere radius:         103.3 mm
Origin head coordinates:      2.4 5.8 40.3 mm
Origin device coordinates:    2.4 5.8 40.3 mm


/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these

Fitted sphere radius:         100.7 mm
Origin head coordinates:      1.1 0.4 42.0 mm
Origin device coordinates:    1.1 0.4 42.0 mm


/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these

Fitted sphere radius:         98.9 mm
Origin head coordinates:      4.5 4.0 40.4 mm
Origin device coordinates:    4.5 4.0 40.4 mm



# Building a dataframe with subjects' electrode positions

In [7]:
locations = []
for subject in trials.subject:
    epochs = get_epochs_lemon(subject.values, lemon_root, event_id)
    eeg_loc = pd.DataFrame({ch["ch_name"]:ch['loc'][:3] for ch in epochs.info['chs']}).T
    eeg_loc.columns = ["x", "y", "z"]
    eeg_loc["subject"] = subject.values
    locations.append(eeg_loc)
locations = pd.concat(locations)

/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these

/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these

/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these

/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these

/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these

/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these

In [8]:
recompute = False
if Path("df_between_csd.pck").exists() and not recompute:
    df_between = pd.read_pickle("df_between.pck")
    df_within = pd.read_pickle("df_within.pck")
    df_within_csd = pd.read_pickle("df_within_csd.pck")
    df_between_csd = pd.read_pickle("df_between_csd.pck")
else:
    df_between = get_coh_dist_df(trials, locations, between=True)
    df_within = get_coh_dist_df(trials, locations)
    df_within_csd = get_coh_dist_df(trials_csd, locations)
    df_between_csd = get_coh_dist_df(trials_csd, locations, between=True)    
    
    df_between.to_pickle("df_between.pck")
    df_within.to_pickle("df_within.pck")
    df_within_csd.to_pickle("df_within_csd.pck")
    df_between_csd.to_pickle("df_between_csd.pck")

# Connectivity on the scalp, sources and simulated EEG

In [ ]:
lambda2s = [1e0, 1e-2, 1e-4, 1e-8]
lambda2s_str = ["1e-0", "1e-2", "1e-4", "1e-8"]
df_within_sim = {}
df_within_csd_sim = {}
df_within_sources = {}
subjects = []
snr = []
np.random.seed(46346)
for lambda2 in tqdm(lambda2s):
    trials_sim = []
    trials_csd_sim = []
    df_within_sources[lambda2] = []
    
    for subject in tqdm(complete_subjects, leave=False): 
        epochs_sim, epochs, df_within_sources_01, stcs_01 = get_epochs_sim_lemon(subject, fwd, lambda2=lambda2, seed=None)
        df_within_sources[lambda2].append(df_within_sources_01)        
        trials_sim.append(get_epochs_to_trials([epochs_sim], subject, use_csd=False))
        trials_csd_sim.append(get_epochs_to_trials([epochs_sim], subject, use_csd=True))
        if lambda2 == 1e-4:
            for epoch_sim, epoch in zip(epochs_sim, epochs):
                snr.append(10*np.log10((np.abs((epoch + epoch_sim)).mean()/2)/np.abs(epoch-epoch_sim).mean()))
                subjects.append(subject)            

    df_within_sim[lambda2] = get_coh_dist_df(xr.concat(trials_sim, dim="subject"), locations)
    df_within_csd_sim[lambda2] = get_coh_dist_df(xr.concat(trials_csd_sim, dim="subject"), locations)
    df_within_sources[lambda2] = pd.concat(df_within_sources[lambda2])
    
snr_df = pd.DataFrame(dict(snr=snr, subject=subjects))    

/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
<ipython-input-2-c47d3aa7e58e>:39: RuntimeWarning: Epochs are not baseline corrected, covariance matrix may be inaccurate
  noise_cov = mne.compute_covariance(epochs, method="auto", verbose=False).as_diag()
<ipython-input-2-c47d3aa7e58e>:40: RuntimeWarning: No average EEG reference present in info["projs"], covariance may be adversely affected. Consider recomputing covariance using with an average eeg reference projector added.
  inv

Not setting metadata
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Loading data for 10 events and 500 original time points ...
0 bad epochs dropped
Fitted sphere radius:         102.9 mm
Origin head coordinates:      0.4 3.3 48.6 mm
Origin device coordinates:    0.4 3.3 48.6 mm


/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
<ipython-input-2-c47d3aa7e58e>:39: RuntimeWarning: Epochs are not baseline corrected, covariance matrix may be inaccurate
  noise_cov = mne.compute_covariance(epochs, method="auto", verbose=False).as_diag()
<ipython-input-2-c47d3aa7e58e>:40: RuntimeWarning: No average EEG reference present in info["projs"], covariance may be adversely affected. Consider recomputing covariance using with an average eeg reference projector added.
  inv

Not setting metadata
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Loading data for 10 events and 500 original time points ...
0 bad epochs dropped
Fitted sphere radius:         99.4 mm
Origin head coordinates:      4.3 5.8 49.5 mm
Origin device coordinates:    4.3 5.8 49.5 mm


/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
<ipython-input-2-c47d3aa7e58e>:39: RuntimeWarning: Epochs are not baseline corrected, covariance matrix may be inaccurate
  noise_cov = mne.compute_covariance(epochs, method="auto", verbose=False).as_diag()
<ipython-input-2-c47d3aa7e58e>:40: RuntimeWarning: No average EEG reference present in info["projs"], covariance may be adversely affected. Consider recomputing covariance using with an average eeg reference projector added.
  inv

...

/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
<ipython-input-2-c47d3aa7e58e>:39: RuntimeWarning: Epochs are not baseline corrected, covariance matrix may be inaccurate
  noise_cov = mne.compute_covariance(epochs, method="auto", verbose=False).as_diag()
<ipython-input-2-c47d3aa7e58e>:40: RuntimeWarning: No average EEG reference present in info["projs"], covariance may be adversely affected. Consider recomputing covariance using with an average eeg reference projector added.
  inv

Not setting metadata
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Loading data for 10 events and 500 original time points ...
0 bad epochs dropped
Fitted sphere radius:         100.2 mm
Origin head coordinates:      6.6 5.4 38.2 mm
Origin device coordinates:    6.6 5.4 38.2 mm


/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
<ipython-input-2-c47d3aa7e58e>:39: RuntimeWarning: Epochs are not baseline corrected, covariance matrix may be inaccurate
  noise_cov = mne.compute_covariance(epochs, method="auto", verbose=False).as_diag()
<ipython-input-2-c47d3aa7e58e>:40: RuntimeWarning: No average EEG reference present in info["projs"], covariance may be adversely affected. Consider recomputing covariance using with an average eeg reference projector added.
  inv

Not setting metadata
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Loading data for 10 events and 500 original time points ...
0 bad epochs dropped
Fitted sphere radius:         99.1 mm
Origin head coordinates:      0.3 5.6 46.1 mm
Origin device coordinates:    0.3 5.6 46.1 mm


/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
/home/christian/code/acareeg/acareeg/lemon.py:58: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(str(Path(root_path) / eeg_rel_path / f"sub-{subject}_{event_id}.set"), verbose=False)
<ipython-input-2-c47d3aa7e58e>:39: RuntimeWarning: Epochs are not baseline corrected, covariance matrix may be inaccurate
  noise_cov = mne.compute_covariance(epochs, method="auto", verbose=False).as_diag()
<ipython-input-2-c47d3aa7e58e>:40: RuntimeWarning: No average EEG reference present in info["projs"], covariance may be adversely affected. Consider recomputing covariance using with an average eeg reference projector added.
  inv

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(5, 2.5), sharey=True, sharex=False)
fig.patch.set_facecolor('white')

dfs = []
for kind, reference, df in zip(["wtn", "wtn", "btwn", "btwn"],
                               ["SCD", "ave", "SCD", "ave"],
                               [df_within_csd, df_within, df_between_csd, df_between]): 
    tmp = df.drop(columns="con").groupby(["ch_names1", "ch_names2"]).mean()
    tmp["ref"] = reference
    tmp["kind"] = kind
    dfs.append(tmp)
dfs = pd.concat(dfs)
dfs["dist"] = np.round(tmp["dist"], -1)    
sns.lineplot(x="dist", y="abs_cohy", hue="kind", style="ref", data=dfs, ax=axes[0])
axes[0].set_xlim([dfs["dist"].min()-5, dfs["dist"].max()+5])

dfs = []
for lambda2, lambda2_str in zip(lambda2s, lambda2s_str):
    for reference, df in zip(["ave", "SCD"], [df_within_sim[lambda2], df_within_csd_sim[lambda2]]): 
        tmp = df.drop(columns="con").groupby(["ch_names1", "ch_names2"]).mean()
        tmp[r"$\lambda^2$"] = lambda2_str
        tmp["ref"] = reference
        dfs.append(tmp)
dfs = pd.concat(dfs)
dfs["dist"] = np.round(tmp["dist"], -1)           
sns.lineplot(x="dist", y="abs_cohy", hue=r"$\lambda^2$", style="ref", data=dfs, ax=axes[1])
axes[1].set_xlim([dfs["dist"].min()-5, dfs["dist"].max()+5])

dfs = []
for lambda2, lambda2_str in zip(lambda2s, lambda2s_str):    
    tmp = df_within_sources[lambda2].drop(columns="con").groupby(["ch_names1", "ch_names2"]).mean()
    tmp[r"$\lambda^2$"] = lambda2_str
    dfs.append(tmp)
dfs = pd.concat(dfs)
dfs["dist"] = np.round(tmp["dist"], -1)           
sns.lineplot(x="dist", y="abs_cohy", hue=r"$\lambda^2$", data=dfs, ax=axes[2])
axes[2].set_xlim([dfs["dist"].min()-5, dfs["dist"].max()+5]) 
    
for ax, title in zip(axes, ["scalp", "simulated", "sources"]):
    ax.legend(ncol=2, prop={'size': 8}, columnspacing=1, borderaxespad=0.4,
              handletextpad=0.4, borderpad=0.2, framealpha=0.3) 
    ax.set_ylabel("Coherence")
    ax.set_title(title, fontsize=10)
    ax.set_xlabel("distance (mm)")
    
fig.tight_layout(rect=[-0.02, -0.04, 1.02, 1.04], h_pad=0.0, w_pad=0)    
ax.set_ylim(0, 1.5)
ax.set_yticks([0, 0.2, 0.4, 0.6, 0.8, 1])

fig.savefig("impact_lambda.png", dpi=300)

In [ ]:
lambda2 = 1e-4
df_within_sources_sur_spa = []
trials_sim_sur_spa = []
trials_csd_sim_sur_spa = []
df_within_sources_sur_pha = []
trials_sim_sur_pha = []
trials_csd_sim_sur_pha = []
np.random.seed(46346)
for subject in tqdm(complete_subjects, leave=False): 
    epochs_sim_sur_spa, epochs_sur_spa, df_within_sources_01_sur_spa, stcs_01_sur_spa \
        = get_epochs_sim_lemon(subject, fwd, lambda2=lambda2, spatial_random=True, seed=None)
    df_within_sources_sur_spa.append(df_within_sources_01_sur_spa)        
    trials_sim_sur_spa.append(get_epochs_to_trials([epochs_sim_sur_spa], subject, use_csd=False))
    trials_csd_sim_sur_spa.append(get_epochs_to_trials([epochs_sim_sur_spa], subject, use_csd=True))
    
    epochs_sim_sur_pha, epochs_sur_pha, df_within_sources_01_sur_pha, stcs_01_sur_pha \
        = get_epochs_sim_lemon(subject, fwd, lambda2=lambda2, phase_random=True, seed=None)
    df_within_sources_sur_pha.append(df_within_sources_01_sur_pha)        
    trials_sim_sur_pha.append(get_epochs_to_trials([epochs_sim_sur_pha], subject, use_csd=False))
    trials_csd_sim_sur_pha.append(get_epochs_to_trials([epochs_sim_sur_pha], subject, use_csd=True))    
    
df_within_sim_sur_spa = get_coh_dist_df(xr.concat(trials_sim_sur_spa, dim="subject"), locations)
df_within_csd_sim_sur_spa = get_coh_dist_df(xr.concat(trials_csd_sim_sur_spa, dim="subject"), locations)
df_within_sources_sur_spa = pd.concat(df_within_sources_sur_spa)  

df_within_sim_sur_pha = get_coh_dist_df(xr.concat(trials_sim_sur_pha, dim="subject"), locations)
df_within_csd_sim_sur_pha = get_coh_dist_df(xr.concat(trials_csd_sim_sur_pha, dim="subject"), locations)
df_within_sources_sur_pha = pd.concat(df_within_sources_sur_pha)

In [ ]:
df_within_intra = pd.read_pickle("within_intracranial.pck")
df_between_intra = pd.read_pickle("between_intracranial.pck")

tmp_bw = df_between_intra.drop(columns="con").groupby(["ch_names1", "ch_names2"]).mean().reset_index()
tmp_bw["dist"] = np.round(tmp_bw["dist"], -1)
dist_bw, abs_cohy_bw = tmp_bw.groupby(["dist"]).mean().reset_index().values.T
offset = abs_cohy_bw[dist_bw < 75].mean()
df_within_intra["abs_cohy"] -= offset

def fit_exp(df, x0):
    def fun(x, df): 
        a, b = x
        return np.abs(np.exp(a*df.dist.values)*b - df.abs_cohy.values).sum()

    df = df.drop(columns="con").sort_values("dist").rolling(50, center=True).mean().dropna()[::50]
    res = minimize(fun, x0, args=(df[df.dist < 75],))
    return res.x[0], res.x[1], df

x0 = np.array([-0.11, 0.2])
a, b, df = fit_exp(df_within_intra, x0)   

fig, ax = plt.subplots(1, 1, figsize=(3.5, 2.0), facecolor='white')
ax.plot(df.dist[df.dist < 75], df.abs_cohy[df.dist < 75], ".")
ax.plot(np.arange(75), np.exp(a*np.arange(75))*b, "k")
ax.set_xlim(3, 30)
ax.set_ylim(-0.02, 0.2)
ax.text(18, 0.17, f"$\sigma(d)={np.round(b, 4)}*exp({np.round(a, 4)})$", ha="center");
ax.set_ylabel("Coherency")
ax.set_xlabel("Distance (mm)")

fig.tight_layout()
fig.savefig("intra_coh_vs_dist.png", dpi=300)

In [ ]:
kdtree = get_model_kdtree(fwd, dist_weight_max=20.0, progress="tqdm_notebook")

In [ ]:
# Compute corrected epochs
c = 0.35
epochs_sim_corrected = {}
correction_amp_factor = {}
df_within_sources_cor = {}
np.random.seed(567456)
for subject in tqdm(complete_subjects, leave=True): 
    epochs = get_epochs_lemon(subject, lemon_root, event_id)
    epochs = epochs[nb_selected_epochs]    
    epochs.load_data()
    stcs = get_sources(epochs, fwd, phase_random=True, seed=None)
    for stc in stcs:
        get_corrected_sources_one_epoch(stc, kdtree, a=a, b=b, c=c, seed=None)
    epochs_sim_corrected[subject], df_within_sources_cor[subject] = get_corrected_amplitude_surrogate(epochs, stcs, fwd, seed=None)
    
df_within_sources_cor = pd.concat(df_within_sources_cor.values())

In [ ]:
corrected_sim_psds = []
recording_psds = []
sur_spa_psds = []
sur_pha_psds = []
np.random.seed(345345)
for subject in tqdm(epochs_sim_corrected):
    corrected_sim_psd, freqs = mne.time_frequency.psd_multitaper(epochs_sim_corrected[subject])
    corrected_sim_psds.append(corrected_sim_psd.mean(0).mean(0))

    epochs = get_epochs_lemon(subject, lemon_root, event_id)
    recording_psds.append(mne.time_frequency.psd_multitaper(epochs)[0].mean(0).mean(0))
    
    epochs_sim_sur_spa = get_epochs_sim_lemon(subject, fwd, lambda2=1e-4, spatial_random=True, seed=None)[0]
    sur_spa_psds.append(mne.time_frequency.psd_multitaper(epochs_sim_sur_spa)[0].mean(0).mean(0))
    
    epochs_sim_sur_pha = get_epochs_sim_lemon(subject, fwd, lambda2=1e-4, phase_random=True, seed=None)[0]
    sur_pha_psds.append(mne.time_frequency.psd_multitaper(epochs_sim_sur_pha)[0].mean(0).mean(0))

In [ ]:
trials_csd_sim_sur_cor = xr.concat([get_epochs_to_trials(sub_trials, subject, use_csd=True) 
                                    for subject, sub_trials in epochs_sim_corrected.items()], dim="subject")
df_within_csd_sim_sur_cor = get_coh_dist_df(trials_csd_sim_sur_cor, locations)

trials_sim_sur_cor = xr.concat([get_epochs_to_trials(sub_trials, subject, use_csd=False) 
                                    for subject, sub_trials in epochs_sim_corrected.items()], dim="subject")
df_within_sim_sur_cor = get_coh_dist_df(trials_sim_sur_cor, locations)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(5, 2.5), sharey=True, sharex=False)
fig.patch.set_facecolor('white')

dfs = []
dfs_sur_cor = [df_within_sim_sur_cor, df_within_csd_sim_sur_cor, df_within_sources_cor]
dfs_sur_spa = [df_within_sim_sur_spa, df_within_csd_sim_sur_spa, df_within_sources_sur_spa]
dfs_sur_pha = [df_within_sim_sur_pha, df_within_csd_sim_sur_pha, df_within_sources_sur_pha]
dfs_originals = [df_within, df_within_csd, df_within_sources[lambda2]]
for kind, dfs_kind in zip(["rec", "spa", "pha", "cor"], [dfs_originals, dfs_sur_spa, dfs_sur_pha, dfs_sur_cor]):
    for ref, df, ax in zip(["ave", "SCD", "sources"], dfs_kind, axes): 
        tmp = df.drop(columns="con").groupby(["ch_names1", "ch_names2"]).mean().reset_index()
        tmp["kind"] = kind
        tmp["ref"] = ref
        tmp["dist"] = np.round(tmp["dist"]*2, -1)/2    
        dfs.append(tmp)
        
df_within_sources_intra = pd.read_pickle("within_intracranial.pck")        
df_within_sources_intra = df_within_sources_intra.drop(columns="con").groupby(["ch_names1", "ch_names2"]).mean().reset_index()
df_within_sources_intra["kind"] = "intra"
df_within_sources_intra["ref"] = "sources"
df_within_sources_intra["dist"] = np.round(df_within_sources_intra["dist"], -1)    
dfs.append(df_within_sources_intra)        
dfs = pd.concat(dfs)

for ref, df, ax in zip(["ave", "SCD", "sources"], dfs_kind, axes): 
    g = sns.lineplot(x="dist", y="abs_cohy", data=dfs[dfs.ref == ref], hue="kind", ax=ax) #ax=axes[0])

for ax, title in zip(axes, ["Average reference", "SCD", "Sources"]):
    ax.set_ylabel("Coherence")
    ax.text(0.5, 0.92, title, ha="center", transform=ax.transAxes)
    ax.set_xlabel("distance (mm)")
    
axes[2].legend()
axes[1].legend().set_visible(False)
axes[0].legend().set_visible(False)

axes[2].set_xlabel("distance (mm)")
axes[2].set_xlim(0, 180)
fig.tight_layout(rect=[-0.0, -0.04, 1.02, 0.92], h_pad=0.0, w_pad=0)    

# Adding and inset
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

axins = axes[2].inset_axes([0.23, 0.4, 0.7, 0.45])
sns.lineplot(x="dist", y="abs_cohy", data=dfs[dfs.ref == "sources"], hue="kind", ax=axins, legend=False)

axins.get_xaxis().set_visible(False)
axins.get_yaxis().set_visible(False)

for spine in axins.spines.values():
    spine.set_edgecolor([0, 0, 0, 0.2])

g.legend(ncol=5, loc='right', bbox_to_anchor=(1.05, 1.1), columnspacing=1.8)    
    
axins.set_ylim(0, 0.2)
axins.set_xlim(0, 50)

fig.savefig("corrected_surrogate.png", dpi=300)

In [ ]:
def get_df_subject_cor_opt(epochs, c=1.0, seed=324234):
    if seed is not None:
        np.random.seed(seed)
    stcs = get_sources(epochs, fwd, phase_random=True, seed=None)
    for stc in stcs:
        get_corrected_sources_one_epoch(stc, kdtree, a=a, b=b, c=c, seed=None)
    return get_corrected_amplitude_surrogate(epochs, stcs, fwd, seed=None)[1]    

np.random.seed(324534)
tmp2s = {}
epochs = get_epochs_lemon(subject, lemon_root, event_id)[nb_selected_epochs].load_data()
for c in tqdm([0.1, 0.2, 0.3, 0.4, 0.5, 0.6]):
    df = get_df_subject_cor_opt(epochs, c=c, seed=None)
    tmp2s[c] = df.drop(columns="con").sort_values("dist").rolling(10, center=True).mean().dropna()[::10].groupby("dist").mean().reset_index()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 2), facecolor='white')

df_within_sources_intra = pd.read_pickle("within_intracranial.pck")  
df_within_sources_intra["dist_bin"] = np.round(df_within_sources_intra.dist*2, -1)/2
tmp1 = df_within_sources_intra.sort_values("dist").rolling(50, center=True).mean().dropna()[::50]
ax.plot(tmp1.dist, tmp1.abs_cohy, ".", label="intra")
ax.plot(np.arange(75), np.exp(a*np.arange(75))*b + offset, "k", linewidth=2, label=r"$\sigma(d)$")

for c in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]:
    tmp2 = tmp2s[c]
    ax.plot(tmp2.dist, tmp2.abs_cohy, label=c)
    
ax.set_xlim(3, 30)
ax.set_ylim(0, 0.20)
ax.legend(ncol=3)
ax.set_ylabel("Coherence")
ax.set_xlabel("Distance (mm)")
ax.text(22.5, 0.075, f"$\sigma(d)={np.round(b, 4)}*exp({np.round(a, 4)})$", ha="center");

fig.tight_layout(rect=[-0.02, -0.06, 1.02, 1.06])
fig.savefig("benchmark_c_value.png", dpi=300)

# Connectivity example

In [ ]:
bands = ["theta", "alpha", "beta", "gamma"]
xr_cons = {}
for type_, trial_lst in zip(["csd", "sim"], [trials_csd, trials_csd_sim_sur_cor]):
    xr_cons[type_] = []
    for subject in tqdm(trial_lst.subject):
        con = mne.connectivity.spectral_connectivity(trial_lst.sel(subject=subject).data, method='cohy', 
                                                     fmin=(4, 8, 12, 30),
                                                     fmax=(8, 12, 30, 100),
                                                     faverage=True,
                                                     sfreq=250, mode='multitaper', verbose=False)[0]
        xr_cons[type_].append(xr.DataArray([con],
                                    dims=("subject", "channel1", "channel2", "band"),
                                    coords={"subject": [str(subject.values)],
                                            "channel1": trial_lst.channel.values,
                                            "channel2": trial_lst.channel.values,
                                            "band": bands}))

    xr_cons[type_] = xr.concat(xr_cons[type_], dim="subject")

In [ ]:
ch_pos = pd.DataFrame(epochs.get_montage().get_positions()["ch_pos"]).T
ch_pos.columns = ["x", "y", "z"]
left = ch_pos[ch_pos["x"] > 0]
right = ch_pos[ch_pos["x"] <= 0]
channels = pd.concat([left.sort_values("y"), right.sort_values("y", ascending=False)]).index.values

channels = np.roll(channels, len(channels) - list(channels).index("Oz"))
channels = [ch for ch in channels if ch not in ['Fz', 'Cz', 'AFz', 'Pz', 'CPz', 'POz']]
angles = np.linspace(360, 0, len(channels), endpoint=False)-90

In [ ]:
figs = {}
axes_lst = {}
figs_im = {}
axes_im_lst = {}
maxima = {}
for i, band in tqdm(list(enumerate(bands))):
    tmp = (xr_cons["csd"].real.mean("subject") - xr_cons["sim"].real.quantile(0.95, "subject"))
    tmp = tmp.sel(channel1=channels, channel2=channels).data[:, :, i]
    tmp[tmp < 0] = 0
    maxima[("real", band)] = tmp.max()    
    figs[i], axes_lst[i] = mne.viz.plot_connectivity_circle(tmp, [""]*len(channels), vmin=0, vmax=tmp.max(),
                                                            node_angles=angles, n_lines=np.sum(tmp > 0),
                                                            colorbar=False, title=f"{band} ({np.round(tmp.max(), 2)})",
                                                            show=False)
    figs[i].savefig(f"{band}_real.png", dpi=300)
    plt.close(figs[i])
    
    tmp = (xr_cons["csd"].imag.mean("subject") - xr_cons["sim"].imag.quantile(0.95, "subject"))
    tmp = tmp.sel(channel1=channels, channel2=channels).data[:, :, i]
    maxima[("imag", band)] = tmp.max()   
    tmp[tmp < 0] = 0
    figs_im[i], axes_im_lst[i] = mne.viz.plot_connectivity_circle(tmp, [""]*len(channels), vmin=0, vmax=tmp.max(),
                                                            node_angles=angles, n_lines=np.sum(tmp > 0),
                                                            colorbar=False, title=f"{band} ({np.round(tmp.max(), 2)})",
                                                            show=False)
    plt.close(figs_im[i])
    figs_im[i].savefig(f"{band}_imag.png", dpi=300)    

In [ ]:
fig = plt.figure(figsize=(5, 3.2), facecolor='white') #constrained_layout=False, 
gs1 = plt.GridSpec(2, 4, hspace=0)        
axes =np.array([[fig.add_subplot(gs1[i, j]) for j in range(4)]  for i in range(2)])
for ax in axes.ravel():
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    ax.set(frame_on=False)
    ax.set_aspect(1)    
gs1.tight_layout(fig, rect=[-0.04, -0.02, 1.02, 1.02], w_pad=0.0, h_pad=0.0)

for type_, ax_row, y_text in zip(["real", "imag"], axes, [0.84, 0.39]):
    for band, ax, x_text in zip(bands, ax_row, [0.17, 0.405, 0.64, 0.875]):
        img = mpimg.imread(f"{band}_{type_}.png")
        margin = 600
        
        ax.imshow(img[300:-margin, margin:-margin])   
        if type_ == "real": 
            ax.set_title(band, fontsize=10)        
            
        fig.text(x_text, y_text, str(np.round(maxima[(type_, band)], 5)), color="w", ha="center") 

fig.text(0.01, 0.62, "Real", rotation=90)
fig.text(0.01, 0.09, "Imaginary", rotation=90)

gs1.tight_layout(fig, rect=[0.02, -0.05, 1.02, 1.02], w_pad=0.0, h_pad=0.0)
fig.savefig("connectivity.png", dpi=300)

# Plot showing the quality of the co-registration

In [ ]:
fig = mne.viz.plot_alignment(epochs.info, src=src, eeg='projected', trans=trans,
                             show_axes=True, mri_fiducials=True, dig='fiducials')


fig.plotter.off_screen = True
fig.plotter.set_background("white")

mne.viz.set_3d_view(figure=fig, azimuth=135, elevation=80, distance=0.6)

fig.plotter.screenshot(f"coregistration_1.png")

mne.viz.set_3d_view(figure=fig, azimuth=45, elevation=80, distance=0.6)
fig.plotter.screenshot(f"coregistration_2.png")

mne.viz.set_3d_view(figure=fig, azimuth=270, elevation=80, distance=0.6)
fig.plotter.screenshot(f"coregistration_3.png");

# Validation of simulation quality

In [ ]:
df_within_sources_01, stcs_01 = get_coh_dist_df_sources(epochs[:2], fwd, 0.1)
raw_sim = mne.simulation.simulate_raw(info=epochs.info, stc=stcs_01, forward=fwd)
nb_epochs = len(stcs_01)
nb_samples = len(epochs.times)
events = np.array([np.arange(0, nb_epochs * nb_samples, nb_samples), [0] * nb_epochs, [1] * nb_epochs]).T
epochs_sim = mne.Epochs(raw_sim, events, tmin=epochs.times[0], tmax=epochs.times[-1], baseline=None, preload=True,
                        reject_by_annotation=False)

In [ ]:
plt.rcParams.update({'font.size': 10})

fig, axes = plt.subplots(1, 2, figsize=(3.5, 2), sharex=True, sharey=True, facecolor='white')

sns.heatmap(epochs[0].get_data().squeeze(), ax=axes[0], cbar_ax=None, cbar=False) #vmin=0, vmax=1
sns.heatmap(epochs_sim[0].get_data().squeeze(), ax=axes[1], cbar_ax=None, cbar=False)
fig.tight_layout(rect=[0, 0, .9, 1])
for ax in axes:
    ax.yaxis.set_ticks([]) 
    ax.xaxis.set_ticks([0, 250, 500])    
    ax.xaxis.set_ticklabels([0, 1, 2], rotation=0)
    ax.set_xlabel("Time (s)")

axes[0].set_ylabel("Channels")
    
axes[0].set_title("Recorded", fontsize=10)
axes[1].set_title("Simulated", fontsize=10)

fig.tight_layout()
fig.savefig("epoch_heatmap.png", dpi=300)

In [ ]:
x = epochs[0].get_data().squeeze()
y = epochs_sim[0].get_data().squeeze()

print(f"SNR = {np.round(10*np.log10((np.abs((x + y)).mean()/2)/np.abs(x-y).mean()), 1)} dB")

In [ ]:
sp.preview(r'$$SNR=10\log_{10}\left( \frac{\overline{|x+y|}}{2\overline{|x-y|}} \right)$$', 
           viewer='file', filename='snr_eq.png', 
           dvioptions=["-D 600"])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(1, 3.5), sharex=True, sharey=True, facecolor='white')
plt.rcParams.update({'font.size': 10})

sns.violinplot(x="snr", y="subject", data=snr_df[np.in1d(snr_df.subject, snr_df.subject.unique()[:10])], orient="h", ax=ax)    
ax.set_xlabel("SNR (dB)")
ax.set_ylabel("")
ax.set_yticks([])
ax.set_xlim(15, 21)

fig.tight_layout(rect=[-0.15, -0.02, 1.1, 1.02])
fig.savefig("simulation_snr.png", dpi=300)

In [ ]:
fig = plt.figure(figsize=(3.5, 3), facecolor='white') #constrained_layout=False, 

gs1 = plt.GridSpec(2, 2, hspace=0, height_ratios=[0.87, 1], width_ratios=[2, 0.7])        
ax1 = fig.add_subplot(gs1[0, 0])
ax2 = fig.add_subplot(gs1[1, 0])
ax3 = fig.add_subplot(gs1[:, 1])
for ax in [ax1, ax2, ax3]:
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
gs1.tight_layout(fig, rect=[-0.04, -0.02, 1.02, 1.02], w_pad=0.0, h_pad=0.0)

margin = 120
img1 = mpimg.imread("coregistration_3.png")[margin:-margin, margin:-margin, :]
img2 = mpimg.imread("coregistration_2.png")[margin:-margin, margin:-margin, :]
img = np.concatenate([img1, img2], axis=1)

ax1.imshow(img)
ax1.set(frame_on=False)
ax1.set_aspect(1)

img = mpimg.imread("epoch_heatmap.png")
ax2.imshow(img)
ax2.set(frame_on=False)
ax2.set_aspect(1)

img = mpimg.imread("simulation_snr.png")
ax3.imshow(img[5:, :, :])
ax3.set(frame_on=False)
ax3.set_aspect(1)

fig.text(0.04, 0.95, "a", size=12, ha="center", transform=fig.transFigure)
fig.text(0.04, 0.45, "b", size=12, ha="center", transform=fig.transFigure)
fig.text(0.72, 0.95, "c", size=12, ha="center", transform=fig.transFigure)

gs1.tight_layout(fig, rect=[-0.04, -0.05, 1.02, 1.02], w_pad=0.0, h_pad=0.0)
fig.savefig("method_paper.png", dpi=300)

In [ ]:
fig = plt.figure(figsize=(3.5, 4.2), facecolor='white') #constrained_layout=False, 

gs1 = plt.GridSpec(2, 1, hspace=0, height_ratios=[1, 1.25])        
ax1 = fig.add_subplot(gs1[0])
ax2 = fig.add_subplot(gs1[1])
for ax in [ax1, ax2]:
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
gs1.tight_layout(fig, rect=[-0.04, -0.02, 1.02, 1.02], w_pad=0.0, h_pad=0.0)

img = mpimg.imread("intra_coh_vs_dist.png")
margin = 120
ax1.imshow(img)
ax1.set(frame_on=False)
ax1.set_aspect(1)

img = mpimg.imread("benchmark_c_value.png")
ax2.imshow(img)
ax2.set(frame_on=False)
ax2.set_aspect(1)

gs1.tight_layout(fig, rect=[-0.04, -0.02, 1.02, 1.02], w_pad=0.0, h_pad=0.0)

fig.text(0.04, 0.97, "a", size=12, ha="center", transform=fig.transFigure)
fig.text(0.04, 0.55, "b", size=12, ha="center", transform=fig.transFigure)

fig.savefig("method_corr_surrogate.png", dpi=300)

# Making equations images for Google doc

In [ ]:
sp.preview(r'$$C(f)=\left | \frac{E\left [ S_{XY}(f))\right ]}{\sqrt{E\left [ S_{XX}(f))\right ]E\left [ S_{YY}(f))\right ]}} \right |$$', 
           viewer='file', filename='abs_coherence_eq.png', 
           dvioptions=["-D 600"])
sp.preview(r'$$\varsigma (d)=be^a=0.3282e^{-0.2261}$$', 
           viewer='file', filename='varsigma_func.png', 
           dvioptions=["-D 600"])
sp.preview(r'$$\widetilde{s_i}= s_i+\sum_{s_j \in \Theta_i }\varsigma (d(s_i, s_j))s_j$$', 
           viewer='file', filename='corrected_source_non_norm_eq.png', 
           dvioptions=["-D 600"])
sp.preview(r'$$\widehat{s_i}= \widetilde{s_i}\frac{\overline{|s_i|}}{\overline{|\widetilde{s_i}|}}$$', 
           viewer='file', filename='corrected_source_normed_eq.png', 
           dvioptions=["-D 600"])